In [ ]:
!pip install torch torchvision -U

In [ ]:
import torch
import os
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
training_path ="/content/drive/MyDrive/train_butterflies/train_split"
test_path  = "/content/drive/MyDrive/valid"

In [ ]:
training_transforms = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])

In [ ]:
train_dataset =torchvision.datasets.ImageFolder(root = training_path, transform =training_transforms)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = 32, shuffle = False)

In [ ]:
def get_mean_and_std(loader):
  mean =0
  std =0
  total_img_count = 0
  for images, _ in loader:
    image_count_in_a_batch = images.size(0)
    images = images.view(image_count_in_a_batch, images.size(1), -1)
    mean+= images.mean(2).sum(0)
    std+= images.std(2).sum(0)
    total_img_count+=image_count_in_a_batch
  mean /= total_img_count
  std /= total_img_count

  return mean, std


In [ ]:
get_mean_and_std(train_loader)

(tensor([0.4731, 0.4613, 0.3259]), tensor([0.2330, 0.2260, 0.2194]))

In [ ]:
mean = [0.4731, 0.4613, 0.3259]
std = [0.2330, 0.2260, 0.2194]

train_transforms = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))])
test_transforms = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))])

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(root = training_path, transform = train_transforms)
test_dataset = torchvision.datasets.ImageFolder(root = test_path, transform = test_transforms)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size =32, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size =32, shuffle = False)

In [ ]:
def set_device():
  if torch.cuda.is_available():
    dev = "cuda:0"
  else:
    dev = "cpu"
  return torch.device(dev)

In [ ]:
def train_nn(model, train_loader, test_loader, criterion, optimiser, n_epochs):
  device = set_device()

  best_acc =0

  for epoch in range(n_epochs):
    print("Epoch number "+ str(epoch+1))
    model.train()
    running_loss =0.0
    running_correct = 0.0
    total = 0

    for data in train_loader:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      total+=labels.size(0)

      optimiser.zero_grad()

      outputs = model(images)

      _, predicted = torch.max(outputs.data, 1)

      loss = criterion(outputs, labels)

      loss.backward()

      optimiser.step()

      running_loss += loss.item()
      running_correct += (labels==predicted).sum().item()
    epoch_loss = running_loss/len(train_loader)
    epoch_acc = 100.0* running_correct/total

    print("   -Training dataset. Got "+ str(running_correct) + " out of " + str(total) + " images correctly (" + str(epoch_acc) + "). Epoch loss: " + str(epoch_loss))

    test_dataset_acc = evaluate_model_on_test_set(model, test_loader)

    if test_dataset_acc > best_acc:
      best_acc = test_dataset_acc
      save_checkpoint(model, epoch, optimiser, best_acc)

  print("Finished")
  return model


In [ ]:
def evaluate_model_on_test_set(model, test_loader):
  model.eval()
  predicted_correctly_on_epoch =0
  total =0
  device = set_device()

  with torch.no_grad():
    for data in test_loader:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      total+=labels.size(0)

      outputs = model(images)

      _, predicted = torch.max(outputs.data, 1)

      predicted_correctly_on_epoch += (labels==predicted).sum().item()

  epoch_acc = 100.0* predicted_correctly_on_epoch/total

  print("     -Testing dataset. Got %d out of %d images correctly (%.3f%%)" % (predicted_correctly_on_epoch, total, epoch_acc))

  return epoch_acc




In [ ]:
def save_checkpoint(model, epoch, optimiser, best_acc):
  state ={
      'epoch': epoch+1,
      'model': model.state_dict(),
      'best_accuracy': best_acc,
      'optimiser': optimiser.state_dict(),

  }
  torch.save(state, 'model_best_checkpoint.pth.tar')

In [ ]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

resnet18_model = models.resnet18(pretrained = True)
num_ftrs = resnet18_model.fc.in_features
number_of_classes = 50
resnet18_model.fc = nn.Linear(num_ftrs, number_of_classes)
device = set_device()
resnet18_model = resnet18_model.to(device)
loss_fn = nn.CrossEntropyLoss()

optimiser = optim.SGD(resnet18_model.parameters(), lr = 0.01, momentum = 0.9, weight_decay = 0.003)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
train_nn(resnet18_model, train_loader, test_loader, loss_fn, optimiser, 5)

Epoch number 1
   -Training dataset. Got 3437.0 out of 4705 images correctly (73.0499468650372). Epoch loss: 1.0666411539187302
     -Testing dataset. Got 216 out of 250 images correctly (86.400%)
Epoch number 2
   -Training dataset. Got 4415.0 out of 4705 images correctly (93.83634431455899). Epoch loss: 0.2560274819717617
     -Testing dataset. Got 219 out of 250 images correctly (87.600%)
Epoch number 3
   -Training dataset. Got 4508.0 out of 4705 images correctly (95.81296493092455). Epoch loss: 0.1640163834357785
     -Testing dataset. Got 228 out of 250 images correctly (91.200%)
Epoch number 4


KeyboardInterrupt: 